## Step-1 Region Proposal

In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

from matplotlib import patches
from PIL import Image
from tqdm import tqdm

In [3]:
# NWPU-RESISC45 Air Plane Dataset
ROOT_DIR = os.path.abspath('./')
DATA_ROOT = os.path.abspath('./data/air_planes')
ANN_DIR = os.path.join(DATA_ROOT, 'annotations/')
IMG_DIR = os.path.join(DATA_ROOT, 'images')

In [4]:
def ApplySelectiveSearch(img):
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(img)
    ss.switchToSelectiveSearchFast()
    ssresults = ss.process()
    #print('Proposed Region :: {}'.format(len(ssresults)))
    
    # ssresults의 bbow 좌표 포맷 x, y, w, h
    # 현재 코드에서 사용되는 bbox 좌표 포맷은 xmin, ymin, xmax, ymax
    # IOU 계산 및  DrawBox 함수 모듈화를 위해 convert
    ssresults[:, 2] = ssresults[:, 0] + ssresults[:, 2]
    ssresults[:, 3] = ssresults[:, 1] + ssresults[:, 3]
        
    return ssresults

In [5]:
def DrawBox(img, bboxes, title='Empty', color='magenta', ax=None):
    if ax is None:
        fig, ax = plt.subplots(1, figsize=(10, 10))
    
    # BBox Display
    # Box 좌표 구성(xmin, ymin, xmax, ymax)
    for bbox in bboxes:
        x1, y1, x2, y2 = bbox
        p = patches.Rectangle((x1, y1), (x2-x1), (y2-y1), linewidth=2, alpha=1.0, linestyle="solid", edgecolor=color, facecolor='none')
        ax.add_patch(p)

    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ax.axis('off')
    ax.set_title(title)

In [6]:
def GetGroundTruthBBox(ann):
    gt_bbox = np.array([], dtype=np.int32).reshape(0, 4)
    for row in ann.iterrows():
        line = row[1][0].split(" ")
        x1 = int(line[0])
        y1 = int(line[1])
        x2 = int(line[2])
        y2 = int(line[3])
        gt_bbox = np.vstack([gt_bbox, [x1, y1, x2, y2]])
    
    return gt_bbox

In [7]:
def ComputeIOU(gt, p):
    x1 = np.maximum(gt[0], p[:, 0])
    y1 = np.maximum(gt[1], p[:, 1])
    x2 = np.minimum(gt[2], p[:, 2])
    y2 = np.minimum(gt[3], p[:, 3])

    intersection = np.maximum(x2 - x1, 0) * np.maximum(y2 - y1, 0)
    gt_area = (gt[2] - gt[0]) * (gt[3] - gt[1])
    propoesed_area = (p[:, 2] - p[:, 0]) * (p[:, 3] - p[:, 1])
    union = gt_area + propoesed_area[:] - intersection[:]

    iou = intersection/union
    
    return iou

In [42]:
def ComputeTargetDelta(gt, p):
    d = np.zeros_like(gt, dtype=np.float32)
    gt = gt.astype(np.float32)
    p = p.astype(np.float32)

    d[0] = ((gt[0] - p[0])/p[2])
    d[1] = (gt[1] - p[1])/p[3]
    d[2] = np.log(gt[2]/p[2])
    d[3] = np.log(gt[3]/p[3])
    
    return d

In [162]:
def WarppingImage(img, pos, neg, delta, cls_trn_img=[], cls_trn_lb=[], reg_trn_img=[], reg_trn_bbox=[]):
    
    cls_trn_img = np.array([], dtype=np.uint8).reshape(0, 224, 224, 3)
    cls_trn_lb = np.array([], dtype=np.int32).reshape(0)
    reg_trn_img = np.array([], dtype=np.uint8).reshape(0, 224, 224, 3)
    reg_trn_delta = np.array([], dtype=np.float32).reshape(0, 4)
    
    for p, d in zip(pos, delta):
        x1, y1, x2, y2 = p
        timg = img[y1:y2, x1:x2]
        rimg = cv2.resize(timg, (224, 224), interpolation = cv2.INTER_AREA)
        
        rimg = np.expand_dims(rimg, axis=0)
        cls_trn_img = np.vstack([cls_trn_img, rimg])
        

    print(cls_trn_img.shape)
        #         cls_trn_img.append(rimg)
#         cls_trn_lb.append(1)
#         reg_trn_img.append(rimg)
#         reg_trn_bbox.append(d)
    
#     for n in neg:
#         x1, y1, x2, y2 = n
#         timg = img[y1:y2, x1:x2]
#         rimg = cv2.resize(timg, (224, 224), interpolation = cv2.INTER_AREA)
#         cls_trn_img.append(rimg)
#         cls_trn_lb.append(0)


In [163]:
def AdjustRegion(gt_bboxes, roi):
    positive = np.array([], dtype=np.int32).reshape(0, 4)
    negative = np.array([], dtype=np.int32).reshape(0, 4)
    delta = np.array([], dtype=np.float32).reshape(0, 4)

    pos_cnt = 0
    neg_cnt = 0
    
    for bbox in gt_bboxes:
        iou_results = ComputeIOU(bbox, roi)

        for idx, iou in enumerate(iou_results):
            if idx < 2000:
                # positive
                if iou >= 0.5 and pos_cnt < 30:
                    positive = np.vstack([positive, roi[idx]])
                    delta = np.vstack([delta, ComputeTargetDelta(bbox, roi[idx])])
                    pos_cnt += 1

                # negative
                elif iou <= 0.3 and neg_cnt < 30:
                    negative = np.vstack([negative, roi[idx]])
                    neg_cnt += 1
            else:
                break

    return positive, negative, delta
            

In [164]:
img_path = os.path.join(DATA_ROOT, IMG_DIR)
ann_path = os.path.join(DATA_ROOT, ANN_DIR)
img_files = sorted(os.listdir(img_path))

cls_trn_img = np.array([], dtype=np.uint8).reshape(0, 224, 224, 3)
cls_trn_lb = np.array([], dtype=np.int32).reshape(0)
reg_trn_img = np.array([], dtype=np.uint8).reshape(0, 224, 224, 3)
reg_trn_delta = np.array([], dtype=np.float32).reshape(0, 4)

is_visible_sample = False

# Annotation File Read
for i, img_file in enumerate(img_files[:1]):
    print(img_file)
    # 1. image file load
    ann_file = '{}.csv'.format(os.path.splitext(img_file)[0])
    img = cv2.imread(os.path.join(img_path, img_file))

    #2. To Obtain positive and negative region
    ## 2-1. Apply Selective Search and obtain ROI
    roi = ApplySelectiveSearch(img)

    ## 2-2. Get Ground Truth Bounding Box Info
    ann = pd.read_csv(os.path.join(ann_path, ann_file))
    gt_bboxes = GetGroundTruthBBox(ann)

    ## 2-3. Generate Train Region to Compute IOU Between GT BBox and ROI
    pos, neg, delta = AdjustRegion(gt_bboxes, roi)

    ## 2-3. Warpping Image
    print('???')
    WarppingImage(img, pos, neg, delta)
    print('!!!')
    
    if i == 0 and is_visible_sample:
        _, ax = plt.subplots(2, 2, figsize=(20, 20))
        DrawBox(img, gt_bboxes, title='GT', ax=ax[0][0])
        DrawBox(img, roi, title='ROI', color='red', ax=ax[0][1])
        DrawBox(img, pos, title='pos', color='blue', ax=ax[1][0])
        DrawBox(img, neg, title='neg', color='cyan', ax=ax[1][1])

        plt.tight_layout()



42845.jpg
???
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(10, 224, 224, 3)
!!!
